In [1]:
import pandas as pd
import numpy as np

In [2]:
#read triple dataset
a=pd.read_csv('qidpidtriples.train.full.2.tsv',sep="\t",names=['qid','positive pid','negative pid'])

In [3]:
#read topics datsaset and training dataset, then merge them according to the query content to get correct 'qid'
topics=pd.read_csv('queries.train.tsv',sep='\t',names=['qid','query'])
train=pd.read_csv('train.csv')
train=pd.merge(train,topics,on='query')

In [4]:
#retrieve the pisitive pairs, drop duplicates and assign them with target=1
b=a[['qid','positive pid']]
b=b.drop_duplicates()
b = b.reindex(columns=['qid', 'positive pid', 'target'],fill_value=1)
b.columns=['qid','pid','target']

In [5]:
#retrieve the negative pairs, drop duplicates and assign them with target=0
c=a[['qid','negative pid']]
# c=c.drop_duplicates()
c=c.reindex(columns=['qid','negative pid','target'],fill_value=0)
c.columns=['qid','pid','target']
c=c.sample(n=len(b)*10)

In [7]:
#concat b and c to get the pair dataset with both positive and negative labels
z=pd.concat([b,c])

In [8]:
#assign 'features' to the collected pairs in dataset 'z'
train=train[['qid_y','docid','features']]
train.columns=['qid','pid','features']
z=pd.merge(z,train,how='inner',on=['pid','qid'])
#resort them by 'qid' to enable svm ranker
z=z.sort_values(by=['qid'])

In [9]:
z=z.astype(str)
with open('triples_train_1:100.txt','a') as f:    
    #retrieve each feature in the list and output to the '.txt' file
    for row in z.iterrows():
        temp=row[-1].iloc[-1]
        features=temp[1:-2].split()

        line=""+row[1].iloc[2]+" qid:"+row[1].iloc[0]+" 1:"+features[0]+" 2:"+features[1]+" 3:"+features[2]+" #docid="+row[1].iloc[1]
        f.writelines(line+"\n")
f.close()

,0 qid:1037798 1:30.83513761 2:8.43367311 3:7.4452028 #docid=8760867
0,2 qid:489204 1:38.86968948 2:12.3242036 3:10.6...
1,0 qid:1113437 1:23.34378542 2:7.72257158 3:6.6...
2,1 qid:130510 1:38.01684238 2:8.61596622 3:7.79...
3,3 qid:833860 1:28.75888371 2:14.27081652 3:12....
4,1 qid:87452 1:24.47173375 2:9.49079543 3:8.418...
...,...
362,3 qid:359349 1:33.4110283 2:5.58237347 3:4.869...
363,3 qid:131843 1:26.19674141 2:6.52329619 3:6.27...
364,0 qid:104861 1:38.02962441 2:11.07139086 3:9.4...
365,2 qid:130510 1:41.33946 2:8.61596622 3:7.79497...
